**Integración Monte Carlo para el Cálculo Paralelo del Momento de Inercia 2D con Densidad Variable**

***Integrantes:***

*   María Estupiñan - 2210727
*   Jefferson Meza - 2231482
*   Fabián Villamizar - 2130438
*   Cristian Ariza - 2221968

Link del Informe: [Aquí](https://github.com/MariaEsA/Algoritmos-y-Arq-Computacionales-de-Alto-Rendimiento/blob/main/parciales/Evaluación%203/Informe%20Ev3.pdf) 🌞
---

In [ ]:
#Librerías a utilizar
from numba import config
import os
os.environ["NUMBA_CUDA_DEFAULT_PTX_CC"] = "7.5"
config.CUDA_ENABLE_PYNVJITLINK = 1
import numpy as np
import time
import math
from numba import njit, prange, cuda

In [ ]:
#Uso de las tarjeta gráfica
!nvidia-smi
from numba import cuda
cuda.runtime.get_version()

Tue Nov 18 14:07:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

(12, 5)

In [ ]:
#Definición física del problema y parte en SERIE
def momento_inercia_mc(num_puntos, radio, densidad_func, eje='y'):
    acum = 0.0
    area_disco = math.pi * radio**2

    for i in range(num_puntos):

        u = np.random.rand()
        v = np.random.rand()
        r = radio * math.sqrt(u)
        theta = 2 * math.pi * v

        x = r * math.cos(theta)
        y = r * math.sin(theta)

        rho = densidad_func(x, y)

        #Calcula la distancia al cuadrado según el eje de rotación
        if eje == 'y':
            distancia2 = x**2
        else:
            distancia2 = y**2

        acum += rho * distancia2

    estim = (area_disco / num_puntos) * acum
    return estim

def crear_densidad_lineal(a, b, c, d, e):
    def densidad(x, y):
        return a + b*x + c*y + d*x**2 + e*y**2
    return densidad


#Versión PARALELA  Numba CPU

@njit(parallel=True)
def _momento_inercia_mc_parallel(u, v, radio, a, b, c, d, e, eje_flag):

    num_puntos = u.shape[0]
    acum = 0.0

    for i in prange(num_puntos):
        r = radio * math.sqrt(u[i])
        theta = 2 * math.pi * v[i]

        x = r * math.cos(theta)
        y = r * math.sin(theta)

        rho = a + b*x + c*y + d*x*x + e*y*y

        if eje_flag == 0:
            distancia2 = x*x   # eje y
        else:
            distancia2 = y*y   # eje x

        acum += rho * distancia2

    return acum

def momento_inercia_mc_parallel(num_puntos, radio, a, b, c, d, e, eje='y'):

    #Generamos los aleatorios en el host (Python)
    u = np.random.rand(num_puntos)
    v = np.random.rand(num_puntos)

    #Codificamos el eje como entero para Numba
    eje_flag = 0 if eje == 'y' else 1

    area_disco = math.pi * radio**2

    #Llamamos al núcleo paralelo
    acum = _momento_inercia_mc_parallel(u, v, radio, a, b, c, d, e, eje_flag)

    estim = (area_disco / num_puntos) * acum
    return estim

#  Versión CUDA (GPU)
@cuda.jit
def kernel_momento_inercia(u, v, radio, a, b, c, d, e, eje_flag, out):
    i = cuda.grid(1)
    if i < u.size:
        r = radio * math.sqrt(u[i])
        theta = 2 * math.pi * v[i]

        x = r * math.cos(theta)
        y = r * math.sin(theta)

        rho = a + b*x + c*y + d*x*x + e*y*y

        if eje_flag == 0:
            distancia2 = x*x   #respecto a eje y
        else:
            distancia2 = y*y   #respecto a eje x

        out[i] = rho * distancia2

def momento_inercia_mc_cuda(num_puntos, radio, a, b, c, d, e, eje='y'):
    #Aleatorios en host
    u = np.random.rand(num_puntos).astype(np.float64)
    v = np.random.rand(num_puntos).astype(np.float64)

    #Eje como entero
    eje_flag = 0 if eje == 'y' else 1
    area_disco = math.pi * radio**2

    #Copiamos a GPU
    d_u = cuda.to_device(u)
    d_v = cuda.to_device(v)
    d_out = cuda.device_array(num_puntos, dtype=np.float64)

    #Configuración de bloques e hilos
    threadsperblock = 256
    blockspergrid = (num_puntos + threadsperblock - 1) // threadsperblock

    #Lanzar kernel
    kernel_momento_inercia[blockspergrid, threadsperblock](
        d_u, d_v, radio, a, b, c, d, e, eje_flag, d_out
    )

    #Vuelve al host y terminar la integral
    out = d_out.copy_to_host()
    acum = out.sum()

    estim = (area_disco / num_puntos) * acum
    return estim

#Programa principal Usuario
if __name__ == "__main__":
    print("Coeficientes de la densidad ρ(x,y) = a + b x + c y + d x² + e y²\n")
    a = float(input("  a (término constante) [default=1] = ") or 1)
    b = float(input("  b (coeficiente de x) [default=0] = ") or 0)
    c = float(input("  c (coeficiente de y) [default=0] = ") or 0)
    d = float(input("  d (coeficiente de x²) [default=2] = ") or 2)
    e = float(input("  e (coeficiente de y²) [default=2] = ") or 2)

    #Función de densidad original (para la versión serial)
    densidad = crear_densidad_lineal(a, b, c, d, e)

    #Parámetros de la simulación
    radio = float(input("\nIngrese el radio del disco [default=1.0]: ") or 1.0)
    num_puntos = int(input("Ingrese número de puntos Monte Carlo [default=2000000]: ") or 2000000)

    eje = 'y'

    print("\n Simulación Monte Carlo SERIE (Python puro)")
    t0 = time.time()
    I_est_serial = momento_inercia_mc(num_puntos, radio, densidad, eje=eje)
    t1 = time.time()
    print(f"  I_est (serial) = {I_est_serial:.6f}")
    print(f"  Tiempo (serial) = {t1 - t0:.3f} s")

    print("\n Simulación Monte Carlo PARALELA (Numba CPU) ")
    t0 = time.time()
    I_est_par = momento_inercia_mc_parallel(num_puntos, radio, a, b, c, d, e, eje=eje)
    t1 = time.time()
    print(f"  I_est (parallel CPU) = {I_est_par:.6f}")
    print(f"  Tiempo (parallel CPU) = {t1 - t0:.3f} s")

    print("\n Simulación Monte Carlo CUDA (GPU) ")
    try:
        t0 = time.time()
        I_est_cuda = momento_inercia_mc_cuda(num_puntos, radio, a, b, c, d, e, eje=eje)
        t1 = time.time()
        print(f"  I_est (CUDA) = {I_est_cuda:.6f}")
        print(f"  Tiempo (CUDA) = {t1 - t0:.3f} s")
    except cuda.cudadrv.error.CudaSupportError as err:
        print("  [CUDA] No hay GPU disponible o no hay soporte CUDA para Numba.")
        print("        Detalle:", err)

Coeficientes de la densidad ρ(x,y) = a + b x + c y + d x² + e y²

  a (término constante) [default=1] = 1
  b (coeficiente de x) [default=0] = 2
  c (coeficiente de y) [default=0] = 1
  d (coeficiente de x²) [default=2] = 1
  e (coeficiente de y²) [default=2] = 8

Ingrese el radio del disco [default=1.0]: 100
Ingrese número de puntos Monte Carlo [default=2000000]: 2000000

 Simulación Monte Carlo SERIE (Python puro)
  I_est (serial) = 1441812753646.343018
  Tiempo (serial) = 3.086 s

 Simulación Monte Carlo PARALELA (Numba CPU) 
  I_est (parallel CPU) = 1439491706312.363525
  Tiempo (parallel CPU) = 0.860 s

 Simulación Monte Carlo CUDA (GPU) 
  I_est (CUDA) = 1439591675617.460449
  Tiempo (CUDA) = 0.397 s
